In [ ]:
# main.py
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from transformers import AutoModelForSemanticSegmentation, AutoImageProcessor
from PIL import Image
import torch
import io
import numpy as np
import uvicorn

# ---------------------------
# 🔧 FastAPI 기본 설정
# ---------------------------
app = FastAPI()

# CORS 설정 → Flutter에서 호출 가능하게
app.add_middleware(
    CORSMiddleware,
    allow_origins=[
        "https://recycleapp.site",    # 네가 배포한 앱 도메인
    ],
    allow_credentials=True,
    allow_methods=["POST"],           # 예측만 할 거니까 POST만
    allow_headers=["Content-Type"],   # JSON 요청만 허용
)

# ---------------------------
# 🔧 모델 로드 (1회만 수행)
# ---------------------------
MODEL_PATH = "C:/Users/USER/Desktop/Recycle_Segmentation/best_model"  

print("🚀 모델 로드 중...")
model = AutoModelForSemanticSegmentation.from_pretrained(MODEL_PATH)
processor = AutoImageProcessor.from_pretrained(MODEL_PATH)
model.eval()  # 평가 모드
print("✅ 모델 로드 완료!")

# ---------------------------
# 🔍 예측 함수
# ---------------------------
def predict_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    
    # 이미지 전처리
    inputs = processor(images=image, return_tensors="pt")

    # 모델 추론
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 결과: 클래스별 softmax → 가장 확률 높은 클래스 추출
    preds = torch.argmax(outputs.logits, dim=1).squeeze().cpu().numpy()

    # numpy 배열을 리스트로 변환해서 반환
    return preds.tolist()

# ---------------------------
# 📤 API 엔드포인트
# ---------------------------
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    image_bytes = await file.read()
    
    # 예측 실행
    prediction = predict_image(image_bytes)

    # 결과 반환
    return {
        "status": "success",
        "prediction": prediction  # 2D 리스트 (segmentation mask)
    }

# ---------------------------
# 🔧 로컬 실행
# ---------------------------
if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)
